In [ ]:
from pax import core, utils, datastructure, simulation, units
from pax.plugins.io.WaveformSimulator import uniform_circle_rv
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rc('font', size=16)
import numpy as np

In [ ]:
# Load the posrec plugins
plugins_to_test = [
    ('MaxPMT', 'PosRecMaxPMT'),
    ('PosSimple', 'PosRecWeightedSum'),
    ('NeuralNet', 'PosRecNeuralNet'),
    ('PosRecChiSquareGamma', 'PosRecChiSquareGamma'),
    ('RobustWeightedMean', 'PosRecRobustWeightedMean')
]
mypax = core.Processor(config_names=['XENON100'], config_dict={
    'pax': {
            'plugin_group_names': ['test'],
            'test':               ["%s.%s" % (a, b) for a, b in plugins_to_test],},
    'PosRecChiSquareGamma.PosRecChiSquareGamma': {
            'seed_from_neural_net': False,
    }
})
sim = mypax.simulator
posrec_plugin = {b: mypax.get_plugin_by_name(b) for a,b in plugins_to_test}

In [ ]:
lcemap = utils.Vector2DGridMap(utils.data_file_name('s2_xy_lce_map_XENON100_Xerawdp0.4.5.json.gz'))

In [ ]:
# Check that sampling algorithm is correct
detector_radius = sim.config['tpc_radius']
xs, ys = uniform_circle_rv(detector_radius, 10000)
plt.figure(figsize=(5,5))
plt.scatter(xs, ys, s=1, edgecolor=None, alpha=0.2)
plt.show()

In [ ]:
n_pe = 1300
n_trials = int(4e4)

# Robustness options
fraction_of_top_randomized = 0
true_gains = np.ones(sim.config['n_channels'])   # Switch to line below this for systematic gain errors
#true_gains = np.random.normal(1, 0.5, sim.config['n_channels'])   # in 'what we think is pe'

# Calculate maximum signal in each channel, assuming hitpattern is capped at 9k pe / channel
# (adjusted for gain variations). 9k is what Sander has on his whiteboard, 
# he studied saturation for radon stuff [TODO: ref note]
max_pe_per_ch = np.array([9e3 / max(1e-9, sim.config['gains'][ch] / 2e6)
                          for ch in range(sim.config['n_channels'])])

gains = np.array(sim.config['gains'])
dead_pmts = np.where(gains == 0)[0]
# Calculate the gain variation per PMT in pe
# Dead pmts give a 'nan' gain sigma: put at mean of others
# Dead PMTs shouldn't get a gain through gain variation error
gain_sigmas = np.array(sim.config['gain_sigmas'])/gains
true_gains[dead_pmts] = 0
gain_sigmas[dead_pmts] = 0
# In case you wanted to resurrect some PMTs which are in fact dead:
# NB the algorithms won't know if you do!
risen_from_dead = gains == 0
true_gains[risen_from_dead] = np.mean(true_gains[True ^ risen_from_dead])
gain_sigmas[risen_from_dead] = np.mean(gain_sigmas[True ^ risen_from_dead])

def get_s2_hitpattern(x, y, n_top, lce_map):
    if n_top == 0:
        return np.zeros(sim.config['n_channels'])
    if lce_map is None:
        ph_per_ch = simulation.randomize_photons_over_channels(
            photon_timings=np.ones(n_top),
            channels=sim.config['channels_top'],
        )
    else:
        ph_per_ch = simulation.distribute_photons_by_lcemap(
            photon_timings=np.ones(n_top), 
            channels=sim.config['channels_top'],
            lce_map=lce_map,
            coordinate_tuple=(x, y)
        )
    return np.array([len(ph_per_ch.get(ch, []))
                     for ch in range(sim.config['n_channels'])])

def euclideandifference(x1, y1, x2, y2):
    return ((x1-x2)**2 + (y1-y2)**2)**0.5

In [ ]:
##
# Run the simulation & reconstruction
##
xs, ys = uniform_circle_rv(detector_radius, n_trials)
rps = {}
differences = {}

for sample_i in tqdm(range(n_trials)):
    x = xs[sample_i]
    y = ys[sample_i]
    
    # How many photons go to the top array? Rest are ignored.
    n_top = np.random.binomial(n=n_pe, p=sim.config['s2_mean_area_fraction_top'])
    
    # Distribute the top array photons
    hitp = np.zeros(sim.config['n_channels'])
    if fraction_of_top_randomized:
        n_random = np.random.binomial(n=n_top, p=fraction_of_top_randomized)
    else:
        n_random = 0
    hitp += get_s2_hitpattern(x, y, n_random, lce_map=None)
    hitp += get_s2_hitpattern(x, y, n_top - n_random, lce_map=lcemap)
    
    # Remove photons from dead PMTs
    hitp[dead_pmts] = 0
    
    # Apply systematic gain errors
    hitp *= true_gains
    
    # Apply gain variation in each PMT
    # Gain variation between PMTs should be corrected for
    hitp += np.random.normal(0, 1, len(hitp)) * gain_sigmas * hitp**0.5

    # Apply saturation
    hitp = np.clip(hitp, 0, max_pe_per_ch) 
    
    e = datastructure.Event.empty_event()
    e.peaks.append(datastructure.Peak({
        'left':  5,
        'right': 9,
        'type':  'S2',
        'detector':  'tpc',
        'area_per_channel': hitp}))

    # Run posrec plugins, collect results
    for pp_name, pp in posrec_plugin.items():
        differences[pp_name] = differences.get(pp_name, [])
        rps[pp_name] = rps.get(pp_name, [])
        e = pp.transform_event(e)
        for peak in e.peaks:
            rp = peak.reconstructed_positions[-1]
            if not rp.algorithm == pp_name:
                print("Algorithm %d dit not give a position!" % rp.algorithm)
            rps[pp_name].append((rp.x, rp.y))
            differences[pp_name].append(
                euclideandifference(rp.x, rp.y, x, y))

# Convert position list of tuples to numpy array
for pp_name, pp in posrec_plugin.items():            
    rps[pp_name] = np.array(rps[pp_name])
    differences[pp_name] = np.array(differences[pp_name])

In [ ]:
# Save to pickle
import pickle
import time
with open('%06dpe_%dtrials_%0.4frandom_%d.pickle' % (n_pe, 
                                                     n_trials,
                                                     fraction_of_top_randomized, 
                                                     time.time()), 
          mode='wb') as outfile:
    pickle.dump(dict(xs=xs, ys=ys,
                     rps=rps, 
                     differences=differences), outfile)

In [ ]:
#Load from pickle
# with open('001300pe_40000trials_0.2000random_1437828569', 'rb') as infile:
#     data = pickle.load(infile)
# xs = data['xs']
# ys = data['ys']
# rps = data['rps']
# differences = data['differences']

In [ ]:
plt.figure(figsize=(10, 7))
for pp_name, pp in posrec_plugin.items():
    plt.hist(differences[pp_name], bins=150, range=(0, 3), histtype='step', label=pp_name)
plt.legend(loc='upper right')
plt.xlabel('Posrec error (cm)')
plt.ylabel('# events')
title = 'Posrec algorithm errors for %d pe S2s' % n_pe
plt.title(title)
#plt.savefig(title)
plt.show()

In [ ]:
# Fiducial volume from xenon:xenon100:analysis:run10:fv, function above fig 3
fiducial_volume_radius = np.sqrt(19481*units.mm**2)

size_multiplier = 1.5

def plot_pmts(pmt_range, color, marker='x', s=20):
    plt.scatter([sim.config['pmt_locations'][i]['x'] for i in pmt_range], 
                [sim.config['pmt_locations'][i]['y'] for i in pmt_range],
                marker='s', s=s, facecolor=color, edgecolors='white')

def plot_top_array():
    plot_pmts([ch for ch in sim.config['channels_top'] if ch in dead_pmts and ch != 0],
              s=40, color='red')
    plot_pmts([ch for ch in sim.config['channels_top'] if ch not in dead_pmts],
              s=40, color='green')

def plot_detector_radius():
    plt.xlim((-1.1*detector_radius, 1.1*detector_radius))
    plt.ylim((-1.1*detector_radius, 1.1*detector_radius))

    plt.gca().add_artist(plt.Circle((0,0), 
                                    detector_radius, 
                                    edgecolor='black', 
                                    fill=None))
    plt.gca().add_artist(plt.Circle((0,0), 
                                    fiducial_volume_radius, 
                                    edgecolor='0.7',
                                    fill=None))

for pp_name in  posrec_plugin.keys():
    plt.figure(figsize=(12*size_multiplier,10*size_multiplier))
    plt.scatter(rps[pp_name][:,0], rps[pp_name][:,1], vmax=2,
                c=differences[pp_name], marker='.', edgecolors='none')
    plt.colorbar(label='Distance from true position (cm)')
    plot_detector_radius()
    plot_top_array()
    title = 'Reconstructed Positions, %dpe S2s, %s' % (n_pe, pp_name)
    plt.title(title)
    #plt.savefig(title)
    plt.show()

In [ ]:
from scipy.stats import binned_statistic_2d
import numpy.ma as ma
import matplotlib.cm as cm

def bin_centers(bin_edges):
    return 0.5*(bin_edges[1:] + bin_edges[:-1])

def twod_stat_plot(x, y, z, statistic='mean', bins=10, range=None, vrange=None, cblabel=None):
    if vrange is None:
        vrange = (None, None)
    result, xbinedges, ybinedges, binnumbers = binned_statistic_2d(
        x, y, z,
        bins=bins,
        statistic=statistic,
        range=range,
    )
    xx, yy = np.meshgrid(xbinedges, ybinedges)    

    Zm = ma.masked_where(np.isnan(result),result)
    plt.figure(figsize=(size_multiplier*12, size_multiplier*10))
    plt.pcolormesh(xx, yy, Zm, vmin=vrange[0], vmax=vrange[1], 
                   #cmap=cm.hot
                  )
    plt.colorbar(label=cblabel)

for pp_name in  posrec_plugin.keys():
    bins = 50
    
    twod_stat_plot(ys, xs, differences[pp_name],
                   bins=bins, vrange=(0, 1),
                   cblabel='Mean posrec error (cm)')
    
    twod_difs = rps[pp_name] - np.array([xs, ys]).T

    x_bias, _, _, _ = binned_statistic_2d(
        xs, ys, twod_difs[:,0],
        bins=bins,
        statistic='mean',
    )
    y_bias, xbinedges, ybinedges, binnumbers = binned_statistic_2d(
        xs, ys, twod_difs[:,1],
        bins=bins,
        statistic='mean',
    )
    
    xx_center, yy_center = np.meshgrid(bin_centers(xbinedges), bin_centers(ybinedges))    
    xx, yy = np.meshgrid(xbinedges, ybinedges)    
    
    plt.quiver(yy_center, xx_center, x_bias, y_bias, 
               angles='xy', scale_units='xy', scale=1, pivot='tail')
    
    plot_top_array()
    plot_detector_radius() 
    title = 'Error by true position, %dpe S2s, %s' % (n_pe, pp_name)
    plt.title(title)
    #plt.savefig(title)

In [ ]:
for pp_name in  posrec_plugin.keys():
    
    rs = np.sqrt(np.array(xs)**2+np.array(ys)**2)
    rs_rec = np.sqrt(rps[pp_name][:,0]**2 + rps[pp_name][:,1]**2)
    radial_errors = rs_rec - rs
    
    plt.figure(figsize=(12, 10))
    
    leak_in =  (rs > fiducial_volume_radius) & (rs_rec < fiducial_volume_radius)
    leak_out = (rs < fiducial_volume_radius) & (rs_rec > fiducial_volume_radius)
    
    plt.scatter(rs[leak_in]**2, 
                radial_errors[leak_in], 
                s=1, alpha=0.5, label='Leak-in', color='red')
    
    plt.scatter(rs[leak_out]**2, 
                radial_errors[leak_out], 
                s=1, alpha=0.5, label='Leak-out', color='blue')
    
    plt.scatter(rs[True ^ (leak_out | leak_in)]**2, 
                radial_errors[True ^ (leak_out | leak_in)], 
                s=1, alpha=0.5)
    
    message = "Leak-in  %0.2f%%\nLeak-out %0.2f%%" % (
            100 * len(np.where(leak_in)[0])/len(np.where(rs > fiducial_volume_radius)[0]),
            100 * len(np.where(leak_out)[0])/len(np.where(rs < fiducial_volume_radius)[0]))
    plt.text(0.05, 0.91, message,
         horizontalalignment='left',
         verticalalignment='center',
         transform = plt.gca().transAxes)
                                                    
    plt.axvline(fiducial_volume_radius**2, color='black', 
                label='Fiducial volume')

    rarg = np.linspace(0, detector_radius, 100)
    error_for_fv = fiducial_volume_radius - rarg    
    plt.plot(rarg**2, error_for_fv, color='red')
    
    plt.xlim(0, 1.02*detector_radius**2)
    #plt.ylim(-3, 1.5)
    plt.ylim(-5, 5)
    plt.xlabel('True radius^2 (cm^2)')
    plt.ylabel('Error on radius (cm)')
    leg = plt.legend(loc='lower left')
    for l in leg.legendHandles:
        l._sizes = [30]
    title = 'Radial errors, %d pe S2s, %s' % (n_pe, pp_name)
    plt.title(title)
    #plt.savefig(title)
    plt.show()